In [1]:
import numpy as np
import pandas as pd
import csv
import sys
import os
  
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
import warnings
import pickle

max_iters = 501
#n_estimators = 10

def logreg(x,y,filename):

   # Model output file name
   file = (os.path.splitext(filename))[0]
   fname = './models/dt_' + file +'/'

   # File for writing precision,recall, f-measure scores for fraud transactions
   f = open('./prf/dt_'+ file + '_prf' +'.txt' ,'w')
   f.write('precision,recall,f-score \n')

   # Stratified sampling based on Y
   X_train, X_test, y_train, y_test = train_test_split(x, y,stratify=y , test_size=0.30, random_state=42)

   # Create 15% validation set and 15% test set split
   X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,stratify=y_test , test_size=0.50, random_state=42)
   
   #Iterations
   it = 1
   
   # Run training algorithm for multiple class weights
   while it < max_iters:
       cw = {}
       cw[0] = 1
       cw[1] = it
       # Train
       print('**************************************')
       print("Iteration number  " , it)
       dt = DecisionTreeClassifier(max_depth=5, random_state=1, class_weight = cw)
       #lr = LogisticRegression(class_weight = cw)
       print('Class weights ', cw)
       #lr.fit(X_train,y_train)
       dt.fit(X_train,y_train)

       # Save trained model to disk
       name = fname + str(cw[1]) + '.sav'
       pickle.dump(dt, open(name, 'wb'))

       # Predict on validation data
       y_val_pred = dt.predict(X_val)
       print('Performance on validation data - Confusion matrix')
       print(confusion_matrix(y_val,y_val_pred))
   
       precision,recall,fscore,support=score(y_val,y_val_pred,average=None)
       print('Precision, Recall, F-score, Support  on validation data' )
       print("Precision" , precision)
       print("Recall" , recall)
       print("F-score" , fscore)
       print("Support" , support)

       p1 = precision[1]
       r1 = recall[1]
       f1 = fscore[1]

       f.write(str(p1) +','+ str(r1) + ',' + str(f1) + '\n') 
       it += 1

   f.close()

def run():
   filename = 'm2.csv'
   df = pd.read_csv(filename, usecols = [9,10,11,12,13,14] , header = 0)
   
   results = list(map(int, df['Anomaly'])) 
   print('Number of fraudulent transactions ' , sum(results))

   features = ['Airfare', 'Lodging', 'Meals', 'Other_Transportation', 'Other_Expenses']
   targets = ['Anomaly']

   # Separating out the features and target variables
   x = df.loc[:, features].values
   y = df.loc[:, targets].values
    
   scaler = preprocessing.StandardScaler()
   x = scaler.fit_transform(x)

   y  = [i for j in y for i in j]
   
   #Ignore warnings
   warnings.filterwarnings("ignore", category=FutureWarning)

   print("***********Logistic Regression**********")
   logreg(x,y,filename)
  
run()

Number of fraudulent transactions  4116
***********Logistic Regression**********
**************************************
Iteration number   1
Class weights  {0: 1, 1: 1}
Performance on validation data - Confusion matrix
[[335006      1]
 [   417    201]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99875679 0.9950495 ]
Recall [0.99999701 0.32524272]
F-score [0.99937652 0.4902439 ]
Support [335007    618]
**************************************
Iteration number   2
Class weights  {0: 1, 1: 2}
Performance on validation data - Confusion matrix
[[335006      1]
 [   396    222]]
Precision, Recall, F-score, Support  on validation data
Precision [0.99881933 0.9955157 ]
Recall [0.99999701 0.3592233 ]
F-score [0.99940782 0.52794293]
Support [335007    618]
**************************************
Iteration number   3
Class weights  {0: 1, 1: 3}
Performance on validation data - Confusion matrix
[[334923     84]
 [   332    286]]
Precision, Recall, F-score, Support  on valida

KeyboardInterrupt: 